In [6]:
import numpy as np
import mne
import moabb
from moabb.datasets import BNCI2014_001
from moabb.paradigms import LeftRightImagery
from moabb.evaluations import WithinSessionEvaluation
import scipy
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from mne.decoding import CSP

class KLDomainAdaptation(BaseEstimator, TransformerMixin):
    def __init__(self, l_max=30, prob_threshold=0.65):
        self.l_max = l_max
        self.prob_threshold = prob_threshold
        self.svm = SVC(probability=True)
        self.cov_train_ = None
        
    def fit(self, X, y):
        # Fit SVM classifier for probability estimation
        self.svm.fit(X, y)
        
        # Calculate training set covariance properly
        X_centered = X - np.mean(X, axis=0)
        self.cov_train_ = np.cov(X_centered, rowvar=False)
        return self
        
    def _calculate_testing_covariance(self, X, j):
        """Calculate mean covariance following equation (10) in the paper"""
        probs = self.svm.predict_proba(X)
        
        # Get indices of samples with high classification probability
        C_l = []  # Left hand samples
        C_r = []  # Right hand samples
        P_l = []  # Left hand probabilities
        P_r = []  # Right hand probabilities
        
        for i, prob in enumerate(probs):
            if prob[0] > self.prob_threshold:  # Left hand
                C_l.append(i)
                P_l.append(prob[0])
            elif prob[1] > self.prob_threshold:  # Right hand
                C_r.append(i)
                P_r.append(prob[1])
                
        N_L = len(C_l)
        N_R = len(C_r)
        
        if N_L + N_R == 0:
            return np.eye(X.shape[1])  # Fallback to identity if no good samples
            
        # Calculate covariance according to equation (10)
        if j < self.l_max:
            X_subset = X
            P_l_subset = P_l
            P_r_subset = P_r
            C_l_subset = C_l
            C_r_subset = C_r
        else:
            start_idx = j - self.l_max
            X_subset = X[start_idx:j]
            P_l_subset = P_l[start_idx:j]
            P_r_subset = P_r[start_idx:j]
            C_l_subset = [idx - start_idx for idx in C_l if idx >= start_idx and idx < j]
            C_r_subset = [idx - start_idx for idx in C_r if idx >= start_idx and idx < j]
            
        # Calculate weighted covariances
        cov_left = np.zeros((X.shape[1], X.shape[1]))
        cov_right = np.zeros((X.shape[1], X.shape[1]))
        
        for idx, prob in zip(C_l_subset, P_l_subset):
            x = X_subset[idx].reshape(-1, 1)
            cov_left += prob * (x @ x.T)
            
        for idx, prob in zip(C_r_subset, P_r_subset):
            x = X_subset[idx].reshape(-1, 1)
            cov_right += prob * (x @ x.T)
            
        return (cov_left + cov_right) / (N_L + N_R)
        
    def _calculate_mapping_F(self, cov_test):
        """Calculate mapping F following equation (8) in the paper"""
        if self.cov_train_ is None:
            raise ValueError("Must call fit before transform")
            
        eps = 1e-10
        cov_train_inv = np.linalg.pinv(self.cov_train_ + eps * np.eye(self.cov_train_.shape[0]))
        F = np.real(scipy.linalg.sqrtm(np.linalg.pinv(cov_train_inv @ cov_test)))
        return F
        
    def transform(self, X):
        if self.cov_train_ is None:
            raise ValueError("Must call fit before transform")
            
        X_new = np.zeros_like(X)
        
        for j in range(X.shape[0]):
            # Calculate testing set covariance
            cov_test = self._calculate_testing_covariance(X[:j+1], j+1)
            
            # Calculate mapping F
            F = self._calculate_mapping_F(cov_test)
            
            # Apply mapping
            X_new[j] = F @ X[j]
            
        return X_new

def create_pipeline():
    """Create pipeline with CSP and KL Domain Adaptation"""
    csp = CSP(n_components=6)
    kl_da = KLDomainAdaptation()
    svm = SVC(kernel='linear', probability=True)
    
    pipeline = Pipeline([
        ('CSP', csp),
        ('KL_DA', kl_da),
        ('SVM', svm)
    ])
    return pipeline

if __name__ == "__main__":
    # Setup dataset
    dataset = BNCI2014_001()
    # dataset.subject_list = [1,2]
    paradigm = LeftRightImagery()
    
    # Evaluation
    evaluation = WithinSessionEvaluation(
        paradigm=paradigm,
        datasets=[dataset],
        overwrite=True
    )
    
    # Create and evaluate pipeline
    pipeline = create_pipeline()
    results = evaluation.process({'KL_DA': pipeline})
    
    # Print results
    print("Classification accuracies:")
    print(results.groupby('subject')['score'].mean())


BNCI2014-001-WithinSession:   0%|          | 0/9 [00:00<?, ?it/s]

No hdf5_path provided, models will not be saved.
Computing rank from data with rank=None


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~

    Using tolerance 32 (2.2e-16 eps * 22 dim * 6.6e+15  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 32 (2.2e-16 eps * 22 dim * 6.6e+15  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 32 (2.2e-16 eps * 22 dim * 6.6e+15  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRI

BNCI2014-001-WithinSession:  11%|█         | 1/9 [00:06<00:55,  6.88s/it]

...

No hdf5_path provided, models will not be saved.
Computing rank from data with rank=None


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~4.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 24 events (all good), 2 – 6 s (baseline off), ~

    Using tolerance 54 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 54 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 55 (2.2e-16 eps * 22 dim * 1.1e+16  max singular value)
    Estimated rank (data): 22
    data: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRI

BNCI2014-001-WithinSession: 100%|██████████| 9/9 [01:01<00:00,  6.86s/it]

Classification accuracies:
subject
1    0.866973
2    0.533980
3    0.930646
4    0.606429
5    0.551735
6    0.629694
7    0.777245
8    0.930884
9    0.732687
Name: score, dtype: float32
